In [2]:
import multiprocessing as mp
import netket as nk
import os
import json
from qutip import *
import numpy as np
import scipy
import cmath
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import itertools
from numpy.random import seed
from numpy.random import rand
from itertools import product
from scipy.misc import derivative
from scipy import optimize
from functools import wraps
from functools import reduce
from time import time
from scipy.optimize import minimize
from matplotlib import gridspec
import copy
import pickle
import optax

In [9]:
eng_list = []
state_list = []

eng_list.append(np.loadtxt('2021_summer_data/ex_hist_var_post_0_excited_N6M6.json')[0][15])
state_list.append(np.loadtxt('2021_summer_data/var_post_0_excited_states.json',dtype=complex)[15])

eng_list.append(np.loadtxt('2021_summer_data/ex_hist_var_post_1_excited_N6M6.json')[0][3])
state_list.append(np.loadtxt('2021_summer_data/var_post_1_excited_states.json',dtype=complex)[3])

eng_list.append(np.loadtxt('2021_summer_data/ex_hist_var_post_2_excited_N6M6.json')[0][11])
state_list.append(np.loadtxt('2021_summer_data/var_post_2_excited_states.json',dtype=complex)[11])

eng_list.append(np.loadtxt('2021_summer_data/ex_hist_var_post_3_excited_N6M6.json')[0][4])
state_list.append(np.loadtxt('2021_summer_data/var_post_3_excited_states.json',dtype=complex)[4])

eng_list.append(np.loadtxt('2021_summer_data/ex_hist_var_post_4_excited_N6M6.json')[0][6])
state_list.append(np.loadtxt('2021_summer_data/var_post_4_excited_states.json',dtype=complex)[6])

In [16]:
data = eng_list
fileName = "2021_summer_data/temp_data/eng_4.json"
np.savetxt(fileName, data)

In [14]:
print(eng_list)

[-1.4152514732915125, -1.3278155640249427, -1.2715227314873638, -1.2198186307873775, -1.1988832538713954]


In [37]:
eng = np.loadtxt("2021_summer_data/temp_data/eng_1.json").tolist()
state = np.loadtxt("2021_summer_data/temp_data/state_1.json", dtype = complex).tolist()

In [5]:
sta = np.loadtxt('2021_summer_data/fail_var_post_5_excited_states.json',dtype=complex)[15]

In [3]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def exactDiagonalization_full(hamiltonian):
    # Changes Hamiltonian to matrix form
    haMatrix = hamiltonian.to_dense()
    # Gets eigenvalues and vectors
    eigenValues, v = np.linalg.eigh(haMatrix)
    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

#input_basis directly from output of exact_diagnolization, input_vec as np array object
def basis_trans(input_vec, input_basis, N):
    vec_new = np.linalg.inv(np.array(v).reshape(2**N, 2**N).T).dot(input_vec.reshape(2**N, 1))
    return vec_new

In [4]:
N = 6
Ak = []

alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
A = N/2
N0 = N/2
for i in range(N-1):
    # Constant A
    #Ak_i = 1
    # Variable A
    Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
    
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)


#Exact Diagonalization
e, v = exactDiagonalization_full(ha)

In [27]:
print('norm of the fifth excited state entry is ' + str(np.abs(basis_trans(sta, v, N)[5])))
print('norm of the sixth excited state entry is ' + str(np.abs(basis_trans(sta, v, N)[6])))

norm of the fifth excited state entry is [0.97652656]
norm of the sixth excited state entry is [0.21506551]


In [35]:
print('norm of the third excited state entry is '+ str(np.abs(basis_trans(state_list[3],v,N)[3])))
print('norm of the fourth excited state entry is '+ str(np.abs(basis_trans(state_list[3],v,N)[4])))

norm of the third excited state entry is [0.9999308]
norm of the fourth excited state entry is [0.00060642]
